# Practica 3
## Maestria en Computo Aplicado

*Topicos de la Industria I*

**Nombre:** Efren Del Real Frias  
**e-mail:** efren.delreal8824@alumnos.udg.mx  
**Fecha:** Abril 15 del 2022

# Software Requirements

* Funciones que generen trayectorias tipo Brownian Motion (BM), Correlated Random Walk (CRW) y Lefy Flight (LF).
Cada una de las funcioones deberá tomar como parámetros el numero de pasos, la velocidad y posición inicial. Además la funciones para CRW  y LF deberán tomar como parámetro el coeficiente para la distribución Cauchy. Por último, la función para LF también deberá aceptar como parámetro el exponente Levy (alpha).
La función retornará un DataFrame de Pandas con todas los puntos que forman la trayectoria.

* El dashboard deberá contener al menos los siguientes elementos (Para generar el dashboard, el estudiante podrá hacer uso de la API de su elección):
* Un panel para desplegar la trayectoria en 3D
* Un panel para desplegar la métrica de elección de la trayectoria.
* Radio Buttons (u otro tipo de selector) que permita elegir el tipo de trayectoria a análizar.
* Widgets que permitan introducir los valores enteros o de punto flotante para ajustar los parámetros de las trayectorias.
Drop down menu (u otro tipo de selector) que permita elegir la métrica a calcualr de la trayectoria bajo análisis.

# Python Header

## Modules

In [1]:
import math
import pandas as pd
import numpy as np

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable
from scipy.spatial import distance

import plotly.graph_objects as go

import panel as pn
pn.extension('plotly')

## Classes

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## Functions

In [83]:
import time
import functools

In [84]:
time.perf_counter()

2659.201272851

In [117]:
globalDelta: float  = 0.0
def measureTimeExe(myFunc):

  @functools.wraps(myFunc)
  def wrapper(*args, **kwargs):
    global globalDelta

    tStart  = time.perf_counter()
    retObj  = myFunc(*args, **kwargs)
    tEnd    = time.perf_counter()

    tDelta = tEnd - tStart

    globalDelta += tDelta

    print(f"Finished {myFunc.__name__!r} in {tDelta:.4f} secs")
    return retObj

  return wrapper

In [163]:
def exeNTimes(myFunc, numTimes, *args, **kwargs):
  global globalDelta

  globalDelta = 0.0
  for _ in range(numTimes):
     retObj = myFunc(*args, **kwargs)


  print(f'Average: {globalDelta/ numTimes}')

  return retObj

    

  

In [178]:
@measureTimeExe
def bm_2d(n_steps=1000, speed=6, s_pos=[0, 0]):
  """
  """
  vVelocity = Vec2d(speed, 0)
  vBM_2d_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos':s_pos[1]}])

  for i in range(n_steps - 1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)

    vVelocity = vVelocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{
        'x_pos': vBM_2d_df.x_pos[i] + vVelocity.x,
        'y_pos': vBM_2d_df.y_pos[i] + vVelocity.y}])
    
    vBM_2d_df =pd.concat([vBM_2d_df, temp_df], ignore_index=True)
  
  return vBM_2d_df

In [188]:
@measureTimeExe
def bm_2d_v2(n_steps=1000, speed=6, s_pos=[0, 0]):
  """
  """
  vVelocity = Vec2d(speed, 0)
  traj = np.empty(shape=(n_steps, 2))

  traj[0][0] = s_pos[0]
  traj[0][1] = s_pos[1]
  

  for i in range(1, n_steps):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)

    vVelocity = vVelocity.rotated(turn_angle)

    traj[i][0] = traj[i - 1][0] + vVelocity.x
    traj[i][1] = traj[i - 1][1] + vVelocity.y

  
  return traj

In [211]:
 df_BM2DTest = exeNTimes(bm_2d, 10)
 df_BM2DTest.shape

Finished 'bm_2d' in 0.9799 secs
Finished 'bm_2d' in 0.9851 secs
Finished 'bm_2d' in 1.1918 secs
Finished 'bm_2d' in 1.0312 secs
Finished 'bm_2d' in 0.9331 secs
Finished 'bm_2d' in 0.9933 secs
Finished 'bm_2d' in 0.9787 secs
Finished 'bm_2d' in 0.9899 secs
Finished 'bm_2d' in 0.9963 secs
Finished 'bm_2d' in 1.1613 secs
Average: 1.0240553315999932


(1000, 2)

In [212]:
 npa_BM2DTest = exeNTimes(bm_2d_v2, 10)
 npa_BM2DTest.shape

Finished 'bm_2d_v2' in 0.0087 secs
Finished 'bm_2d_v2' in 0.0075 secs
Finished 'bm_2d_v2' in 0.0059 secs
Finished 'bm_2d_v2' in 0.0059 secs
Finished 'bm_2d_v2' in 0.0062 secs
Finished 'bm_2d_v2' in 0.0063 secs
Finished 'bm_2d_v2' in 0.0061 secs
Finished 'bm_2d_v2' in 0.0065 secs
Finished 'bm_2d_v2' in 0.0062 secs
Finished 'bm_2d_v2' in 0.0059 secs
Average: 0.006518041000163066


(1000, 2)

In [213]:
figTraj = go.Figure()

figTraj.add_trace(
    go.Scatter3d(
        x=df_BM2DTest.x_pos,
        y=df_BM2DTest.y_pos,
        z=df_BM2DTest.index,
        marker=dict(size=2),
        line=dict(color='blue', width=2),
        mode='lines',
        name=f'steps = {df_BM2DTest.shape[0]}',
        showlegend=True
      )
)

figTraj.show()

In [214]:
figTraj = go.Figure()

figTraj.add_trace(
    go.Scatter3d(
        x=npa_BM2DTest[:,0],
        y=npa_BM2DTest[:,1],
        z=np.arange(len(npa_BM2DTest)),
        marker=dict(size=2),
        line=dict(color='blue', width=2),
        mode='lines',
        name=f'steps = {len(npa_BM2DTest)}',
        showlegend=True
      )
)

figTraj.show()

## Turning angle

In [67]:
###############################################################################################
# Turning angle
###############################################################################################
def turning_angle(vec_a, vec_b, vec_c):
    """
    Arguments:
        vec_a: First detection coordinates
        vec_b: Second detection coordinates
        vec_c: Third detection coordinates
    Returns:
        theta: Turning angle 
    """
    ab = vec_b-vec_a
    norm_ab = np.linalg.norm(ab)
    
    bc = vec_c-vec_b
    norm_bc = np.linalg.norm(bc)

    dot_p = np.dot(ab, bc)
    
    cross_p = np.cross(ab, bc)
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1
    
    cos_theta = dot_p/(norm_ab*norm_bc+np.finfo(float).eps)
    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta

$$
Euclidean(A, B) = \sqrt{(x_{2} - x_{1})^2  + (y_{2} - y_{1})^2}
$$

$where:$  


*   $A(x_{1}, y_{1})$
*   $B(x_{2}, y_{2})$




In [119]:
###############################################################################################
# Compute Path Lenght
###############################################################################################
@measureTimeExe
def compute_path_lenght(rw_2d_df):
  """
  Arguments:
        rw_2d_df: pd.DataFrame - Random Walk pandas DataFrame size(rows = n, columns = 2)

  Returns:
        pl:       np.array     - Path Lenght numpy array size(rows = n, columns = 1)

  See:
        https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.euclidean.html
  """
  disEuclidean = np.array([ distance.euclidean( rw_2d_df.iloc[i - 1], rw_2d_df.iloc[i])
                            for i in range(1, rw_2d_df.shape[0]) ])
  pl = np.cumsum(disEuclidean)

  return pl

In [215]:
npa_BM2DTest.shape

(1000, 2)

In [218]:
npa_BM2DTest[0:5]

array([[  0.        ,   0.        ],
       [ -5.51670268,   2.35923538],
       [ -3.2994602 ,  -3.21605258],
       [ -9.29579091,  -3.0062477 ],
       [-13.70387224,   1.06423372]])

In [219]:
npa_BM2DTest[3]

array([-9.29579091, -3.0062477 ])

In [220]:
###############################################################################################
# Compute Path Lenght v2
###############################################################################################
@measureTimeExe
def compute_path_lenght_v2(npa_rw2D):
  """
  Arguments:
        npa_rw2D: np.array - Random Walk pandas DataFrame size(rows = n, columns = 2)

  Returns:
        pl:       np.array     - Path Lenght numpy array size(rows = n, columns = 1)

  See:
        https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.euclidean.html
  """
  disEuclidean = np.array([ distance.euclidean( npa_rw2D[i - 1], npa_rw2D[i])
                            for i in range(1, npa_rw2D.shape[0]) ])
  pl = np.cumsum(disEuclidean)

  return pl

In [226]:
try:
  pl1 = exeNTimes(compute_path_lenght, 10, df_BM2DTest)
  print(pl1.shape)
except NameError:
  print('DataFrame is no defined yet')

Finished 'compute_path_lenght' in 0.1613 secs
Finished 'compute_path_lenght' in 0.1976 secs
Finished 'compute_path_lenght' in 0.3533 secs
Finished 'compute_path_lenght' in 0.3385 secs
Finished 'compute_path_lenght' in 0.1632 secs
Finished 'compute_path_lenght' in 0.1648 secs
Finished 'compute_path_lenght' in 0.2036 secs
Finished 'compute_path_lenght' in 0.1772 secs
Finished 'compute_path_lenght' in 0.1783 secs
Finished 'compute_path_lenght' in 0.1814 secs
Average: 0.21191785840001104
(999,)


In [227]:
try:
  pl2 = exeNTimes(compute_path_lenght_v2, 10, npa_BM2DTest)
  print(pl2.shape)
except NameError:
  print('DataFrame is no defined yet')

Finished 'compute_path_lenght_v2' in 0.0156 secs
Finished 'compute_path_lenght_v2' in 0.0111 secs
Finished 'compute_path_lenght_v2' in 0.0113 secs
Finished 'compute_path_lenght_v2' in 0.0118 secs
Finished 'compute_path_lenght_v2' in 0.0117 secs
Finished 'compute_path_lenght_v2' in 0.0117 secs
Finished 'compute_path_lenght_v2' in 0.0130 secs
Finished 'compute_path_lenght_v2' in 0.0113 secs
Finished 'compute_path_lenght_v2' in 0.0149 secs
Finished 'compute_path_lenght_v2' in 0.0166 secs
Average: 0.012895727299746796
(999,)


### 3 mean_squared_displacement

$$
MSD = \frac{1}{N - n} \sum_{i=1}^{N-n}(r_{i+n} -r_{i})^2
$$

where $n = 1,...., N - 1$

In [6]:
###############################################################################################
# Compute Path Lenght
###############################################################################################
def mean_squared_displacement(rw_2d_df, tau: int) -> float:
  """
  Arguments:
        rw_2d_df: pd.DataFrame - Random Walk pandas DataFrame size(rows = n, columns = 2)
        tau:      integer      - iterable number to displace two points (called "n" in above equation)
  Returns:
        msd:      float        - mean squared displacement value
  """
  displacement = np.array([ distance.euclidean( rw_2d_df.iloc[i - tau], rw_2d_df.iloc[i] ) 
                            for i in range(tau, rw_2d_df.shape[0]) ])
  
  msd: float = np.mean(np.power(displacement, 2))
  
  return msd

In [159]:
@measureTimeExe
def getMSD(df_rw):
  metric = np.empty(shape=(0))
  for tau in range(1,df_rw.shape[0]):
      metric = np.concatenate( (metric, [mean_squared_displacement(df_rw, tau)]), axis=0)
  return metric

In [228]:
###############################################################################################
# Compute Path Lenght v2
###############################################################################################
def mean_squared_displacement_v2(npa_rw2D, tau: int) -> float:
  """
  Arguments:
        rw_2d_df: np.array     - Random Walk pandas DataFrame size(rows = n, columns = 2)
        tau:      integer      - iterable number to displace two points (called "n" in above equation)
  Returns:
        msd:      float        - mean squared displacement value
  """
  displacement = np.array([ distance.euclidean( npa_rw2D[i - tau], npa_rw2D[i] ) 
                            for i in range(tau, npa_rw2D.shape[0]) ])
  
  msd: float = np.mean(np.power(displacement, 2))
  
  return msd

In [229]:
@measureTimeExe
def getMSD_v2(npa_rw):
  metric = np.empty(shape=(0))
  for tau in range(1,npa_rw.shape[0]):
      metric = np.concatenate( (metric, [mean_squared_displacement_v2(npa_rw, tau)]), axis=0)
  return metric

In [233]:
@measureTimeExe
def getMSD_v3(npa_rw):
  rowEle = npa_rw.shape[0]
  metric = np.empty(shape=(rowEle,))

  for tau in range(1, rowEle):
    metric[ tau - 1 ] = mean_squared_displacement_v2(npa_rw, tau)
  return metric

In [236]:
@measureTimeExe
def getMSD_v4(npa_rw):

  metric = np.array([mean_squared_displacement_v2(npa_rw, tau) for tau in range(1, npa_rw.shape[0])])

  return metric

In [230]:
try:
  msd1 = exeNTimes(getMSD, 10, df_BM2DTest)
except NameError:
  print('DataFrame is no defined yet')

Finished 'getMSD' in 104.2457 secs
Finished 'getMSD' in 99.0401 secs
Finished 'getMSD' in 99.9159 secs
Finished 'getMSD' in 101.6015 secs
Finished 'getMSD' in 99.8245 secs
Finished 'getMSD' in 100.4493 secs
Finished 'getMSD' in 100.4529 secs
Finished 'getMSD' in 99.4015 secs
Finished 'getMSD' in 99.4814 secs
Finished 'getMSD' in 99.7554 secs
Average: 100.41682486030004


In [234]:
try:
  msd2 = exeNTimes(getMSD_v2, 10, npa_BM2DTest)
except NameError:
  print('DataFrame is no defined yet')

Finished 'getMSD_v2' in 6.2141 secs
Finished 'getMSD_v2' in 6.1844 secs
Finished 'getMSD_v2' in 6.1955 secs
Finished 'getMSD_v2' in 6.4703 secs
Finished 'getMSD_v2' in 6.1913 secs
Finished 'getMSD_v2' in 6.1917 secs
Finished 'getMSD_v2' in 6.2523 secs
Finished 'getMSD_v2' in 6.1254 secs
Finished 'getMSD_v2' in 6.1454 secs
Finished 'getMSD_v2' in 6.8764 secs
Average: 6.284677448500224


In [235]:
try:
  msd3 = exeNTimes(getMSD_v3, 10, npa_BM2DTest)
except NameError:
  print('DataFrame is no defined yet')

Finished 'getMSD_v3' in 6.6961 secs
Finished 'getMSD_v3' in 6.3951 secs
Finished 'getMSD_v3' in 7.3642 secs
Finished 'getMSD_v3' in 6.5447 secs
Finished 'getMSD_v3' in 6.4143 secs
Finished 'getMSD_v3' in 6.7640 secs
Finished 'getMSD_v3' in 6.3814 secs
Finished 'getMSD_v3' in 6.4299 secs
Finished 'getMSD_v3' in 6.6506 secs
Finished 'getMSD_v3' in 6.1511 secs
Average: 6.579150068300078


In [244]:
try:
  msd4 = exeNTimes(getMSD_v4, 10, npa_BM2DTest)
except NameError:
  print('DataFrame is no defined yet')

Finished 'getMSD_v4' in 6.4910 secs
Finished 'getMSD_v4' in 6.5087 secs
Finished 'getMSD_v4' in 7.0350 secs
Finished 'getMSD_v4' in 6.4814 secs
Finished 'getMSD_v4' in 6.4031 secs
Finished 'getMSD_v4' in 6.4527 secs
Finished 'getMSD_v4' in 6.9720 secs
Finished 'getMSD_v4' in 6.4562 secs
Finished 'getMSD_v4' in 6.4550 secs
Finished 'getMSD_v4' in 6.8297 secs
Average: 6.608481274999758


In [240]:
msd4.shape

(999,)

In [243]:
# Init figure
fig_path_length = go.Figure()

timePlot = np.arange(len(msd4))
# first trace MSD_BM
## start - Add your code here
fig_path_length.add_trace(
    go.Scatter(
        x=timePlot,
        y=msd2[:],
        marker=dict(size=2),
        line = dict(width=2),
        mode = 'lines',
        name = 'MSD BM 6',
        showlegend = True
))       
## end - Add your code here


# Second trace MSD_CRW
## start - Add your code here
fig_path_length.add_trace(
    go.Scatter(
        x=timePlot,
        y=msd3[:],
        marker=dict(size=2),
        line = dict(width=2),
        mode = 'lines',
        name = 'MSD CRW 6 c=0.9',
        showlegend = True
))         
## end - Add your code here

# Second trace MSD_CRW
## start - Add your code here
fig_path_length.add_trace(
    go.Scatter(
        x=timePlot,
        y=msd4[:],
        marker=dict(size=2),
        line = dict(width=2),
        mode = 'lines',
        name = 'MSD CRW 6 c=0.9',
        showlegend = True
))         
## end - Add your code here

fig_path_length.show()

## 4 Levy

In [7]:
###############################################################################################
# Compute Path Lenght
###############################################################################################
def levy_compute_step_lenght(levy_df):
  """
  Arguments:
        levy_df:                    pd.DataFrame - 
        Levy Walk pandas DataFrame size(rows = n, columns = 2)

  Returns:
        (aux_ta_Levy, aux_sl_Levy):  Tuple(np.array, np.array) - 
        Levy angle and Levy step lengh 
  """
  # Define Local Variables
  theta:        float = 0.0        
  prev_theta:   float = (3 * np.pi)  #
  prev_index:     int = 0
  cal_vel:       bool = True         # Calculate Velocity
  velocity:     float = 0.0

  # aux to store turning angles
  aux_ta_Levy = np.empty(shape=(0))
  # aux to store step-lengths
  aux_sl_Levy = np.empty(shape=(0))

  # Start to calculate over all DataFrame row values
  for index, _ in levy_df[1:-1].iterrows():

      a = levy_df.iloc[index - 1]
      b = levy_df.iloc[index]
      c = levy_df.iloc[index + 1]

      theta = turning_angle(a, b, c)
      
      # Ignore 0.0 since is still in the same direction
      if (theta != 0.0):

        if (prev_theta !=  theta):
          
          # Add to the end the new direction
          aux_ta_Levy = np.concatenate( (aux_ta_Levy, [theta]), axis=0)

          # Gets path lenght - This value includes velocity
          pl = distance.euclidean( levy_df.iloc[prev_index] , levy_df.iloc[index] )

          # Add to the end new path lenght
          pl = np.round(pl)
          aux_sl_Levy = np.concatenate ( (aux_sl_Levy, [pl]) , axis=0)


          # Keeps current index and angle to the next step
          prev_index = index
          prev_theta = theta
      else:
        # theta == 0.0
        if cal_vel:
          # Gets velocity from two close steps
          velocity = distance.euclidean( levy_df.iloc[prev_index] , levy_df.iloc[index] )
          
          # Clean in order to only execute one time
          cal_vel = False

  
  if not cal_vel:
    # We need to divite over the velocity in order
    # to get Levy step lenght
    aux_sl_Levy = aux_sl_Levy / velocity

  return aux_ta_Levy, aux_sl_Levy

## Widgets

In [8]:
wgTrajType = pn.widgets.RadioButtonGroup(
    name='Trajectory type', options=['BM', 'CRW', 'LF'], 
    button_type='default' # [default, primary, success, warning, danger]
    )


wgTrajType

RadioButtonGroup(name='Trajectory type', options=['BM', 'CRW', 'LF'], value='BM')

In [9]:
wgMtcrType = pn.widgets.Select(value='PL', options=['PL','MSD'], name='Metrics type')
wgMtcrType

Select(name='Metrics type', options=['PL', 'MSD'], value='PL')

In [10]:
wgNsteps = pn.widgets.IntSlider(name='number of steps', start=50, end=1000, step=50, value=200)

wgNsteps

IntSlider(end=1000, name='number of steps', start=50, step=50, value=200)

In [11]:
wgSpeed = pn.widgets.IntSlider(name='Speed', start=1, end=10, step=1, value=4)

wgSpeed

IntSlider(end=10, name='Speed', start=1, value=4)

In [12]:
wgXInitPos = pn.widgets.IntInput(name='xInitPos', value=1, step=1, start=0, end=100)
wgXInitPos

IntInput(end=100, name='xInitPos', start=0, value=1)

In [13]:
wgYInitPos = pn.widgets.IntInput(name='yInitPos', value=0, step=1, start=0, end=100)
wgYInitPos

IntInput(end=100, name='yInitPos', start=0)

In [14]:
@pn.depends(wgXInitPos, wgYInitPos)
def display_InitPos(xVal, yVal):
  return (xVal, yVal)

In [15]:
pn.Row(wgXInitPos, wgYInitPos, display_InitPos)

Row
    [0] IntInput(end=100, name='xInitPos', start=0, value=1)
    [1] IntInput(end=100, name='yInitPos', start=0)
    [2] ParamFunction(function)

# Implementation

In [16]:
# Init eGererate Random Walk DataFrame
df_genrw = pd.DataFrame([{'x_pos':  [], 'y_pos': []}])


@pn.depends(wgTrajType, wgNsteps, wgSpeed, wgXInitPos, wgYInitPos)
def main_plotTraj(
    trajType: str,
    Nsteps:   int, 
    speedVal: int, 
    xPos:     int, 
    yPos:     int):
  """
  """

  #
  global df_genrw

  #
  figTraj = go.Figure()

  #
  if ('BM' == trajType):
    pass
  elif ('CRW' == trajType):
    pass
  elif ('LF' == trajType):
    pass

  #
  df_genrw = bm_2d(n_steps=Nsteps, speed=speedVal , s_pos=[xPos, yPos])

  # Plot trajectory in 3D space
  figTraj.add_trace(
      go.Scatter3d(
          x=df_genrw.x_pos,
          y=df_genrw.y_pos,
          z=df_genrw.index,
          marker=dict(size=2),
          line=dict(color='blue', width=2),
          mode='lines',
          name=f'steps = {Nsteps}',
          showlegend=True
      )
  )

  return figTraj

In [17]:
type(df_genrw)

pandas.core.frame.DataFrame

In [18]:
@pn.depends(wgMtcrType)
def main_plotMtcr(mtcrType: str):

  #
  global df_genrw

  #
  metric = np.empty(shape=(0))
  fig_mctr = go.Figure()



  #  
  if ('PL' == mtcrType):
    metric   = compute_path_lenght(df_genrw)
  elif ('MSD' == mtcrType):
    for tau in range(1,df_genrw.shape[0]):
      metric = np.concatenate( (metric, [mean_squared_displacement(df_genrw, tau)]), axis=0)



  # Plot trajectory in 3D space
  fig_mctr.add_trace(
      go.Scatter(
          x=np.arange(len(metric)),
          y=metric,
          marker=dict(size=2),
          line = dict(width=2),
          mode = 'lines',
          name = f'{mtcrType}',
          showlegend = True
  ))

  return fig_mctr

In [19]:
# Dashboard
wgPanel = pn.Row(
    pn.Column(
        'A',
        wgTrajType, 
        wgNsteps,
        pn.Row(wgXInitPos, wgYInitPos),
        wgSpeed),
    pn.Column(
        '3D Trajectory',
        main_plotTraj),
    pn.Column(
        'Metrics',
        wgMtcrType,
        main_plotMtcr)
)
wgPanel

Row
    [0] Column
        [0] Markdown(str)
        [1] RadioButtonGroup(name='Trajectory type', options=['BM', 'CRW', 'LF'], value='BM')
        [2] IntSlider(end=1000, name='number of steps', start=50, step=50, value=200)
        [3] Row
            [0] IntInput(end=100, name='xInitPos', start=0, value=1)
            [1] IntInput(end=100, name='yInitPos', start=0)
        [4] IntSlider(end=10, name='Speed', start=1, value=4)
    [1] Column
        [0] Markdown(str)
        [1] ParamFunction(function)
    [2] Column
        [0] Markdown(str)
        [1] Select(name='Metrics type', options=['PL', 'MSD'], value='PL')
        [2] ParamFunction(function)